In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
#Se cargan las reviews de Darden
path_data = '../data/clean'
data_darden = pd.read_parquet(f'{path_data}/reviews_darden.parquet')

In [6]:
data_darden.sample()

,name,review_id,user_id,business_id,stars,text,date,day,text_reply,city,state,postal_code,coordinates
2441,LongHorn Steakhouse,Us8-iMuQpX28zx_DS1JDIA,ou6T6K7vDEveO7BXMl1N0w,epVHdJqIFN0dtAiFvnVJVg,3.0,"Longhorn, being the only chain steakhouse by o...",2011-02-17,Thursday,,Philadelphia,PA,19114,"40.0844101,-75.0240676748"


In [12]:
reviews_darden_negativas = data_darden[data_darden.stars <= 3][['review_id','date','name','stars','text']]
reviews_darden_negativas.sample()

,review_id,date,name,stars,text
6190,ljYOzT50q84e-UQSPYm5dA,2017-12-30,Cheddar's Scratch Kitchen,1.0,Mediocre food. Blackened Redfish came on a col...


In [15]:
with open('modelo_ML_negativas2.pkl', 'rb') as archivo:
    modelo_ml = pickle.load(archivo)

In [22]:
def df_reviews_etiquetadas(df_reviews, modelo_ml):
    """
    """
    problematicas = [modelo_ml.predict([review])[0] for review in df_reviews['text']]
    
    df_reviews_ml = pd.DataFrame({
        'review_id': df_reviews['review_id'],
        'date': df_reviews['date'],
        'name': df_reviews['name'],
        'text': df_reviews['text'],
        'problem': problematicas
    })

    df_problematicas = df_reviews_ml['problem'].apply(pd.Series)
    df_problematicas.rename(columns={0: 'bad_food', 1: 'cost', 2: 'filthy', 3: 'missing_food', 4: 'order_problem', 5: 'rude_service', 6: 'bad_neighborhood', 7: 'slow_service'}, inplace=True)
    df_reviews_negativas_etiquetadas = pd.concat([df_reviews[['review_id', 'date','name','text']], df_problematicas], axis=1)
    df_reviews_negativas_etiquetadas['date'] = pd.to_datetime(df_reviews_negativas_etiquetadas['date'])

    
    df_reviews_negativas_etiquetadas['quarter'] = df_reviews_negativas_etiquetadas['date'].dt.to_period('Q')

    # Se por trimestre y restaurante, y crear un diccionario las reseñas y sus problematicas
    df_reviews_negativas_etiquetadas_agrupadas = df_reviews_negativas_etiquetadas.groupby(['name', 'quarter']).apply(lambda x: {
        'review_id': x['review_id'].count(),
        'bad_food': sum(x['bad_food'].tolist()),
        'cost': sum(x['cost'].tolist()),
        'filthy': sum(x['filthy'].tolist()),
        'missing_food': sum(x['missing_food'].tolist()),
        'order_problem': sum(x['order_problem'].tolist()),
        'rude_service': sum(x['rude_service'].tolist()),
        'bad_neighborhood': sum(x['bad_neighborhood'].tolist()),
        'slow_service': sum(x['slow_service'].tolist())
    }).unstack()
    return df_reviews_negativas_etiquetadas_agrupadas.to_dict()


In [23]:
df_reviews_negativas_etiquetadas = df_reviews_etiquetadas(reviews_darden_negativas, modelo_ml)

/tmp/ipykernel_152254/1589294117.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_reviews_negativas_etiquetadas_agrupadas = df_reviews_negativas_etiquetadas.groupby(['name', 'quarter']).apply(lambda x: {


In [24]:
df_reviews_negativas_etiquetadas

{Period('2006Q3', 'Q-DEC'): {'Bahama Breeze': {'review_id': 1,
   'bad_food': 0,
   'cost': 0,
   'filthy': 0,
   'missing_food': 0,
   'order_problem': 0,
   'rude_service': 0,
   'bad_neighborhood': 0,
   'slow_service': 0},
  "Cheddar's Scratch Kitchen": nan,
  "Eddie V's Prime Seafood": nan,
  'LongHorn Steakhouse': nan,
  'Olive Garden Italian Restaurant': {'review_id': 1,
   'bad_food': 0,
   'cost': 0,
   'filthy': 0,
   'missing_food': 0,
   'order_problem': 0,
   'rude_service': 0,
   'bad_neighborhood': 0,
   'slow_service': 0},
  "Ruth's Chris Steak House": nan,
  'Seasons 52': nan,
  'Yard House': nan},
 Period('2007Q2', 'Q-DEC'): {'Bahama Breeze': nan,
  "Cheddar's Scratch Kitchen": nan,
  "Eddie V's Prime Seafood": nan,
  'LongHorn Steakhouse': {'review_id': 1,
   'bad_food': 1,
   'cost': 0,
   'filthy': 0,
   'missing_food': 0,
   'order_problem': 0,
   'rude_service': 0,
   'bad_neighborhood': 0,
   'slow_service': 0},
  'Olive Garden Italian Restaurant': nan,
  "Ruth'